In [1]:
! pip install textattack  sentence_transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 4.1 MB/s 
     |████████████████████████████████| 85 kB 2.5 MB/s 
     |████████████████████████████████| 72 kB 580 kB/s 
     |████████████████████████████████| 125 kB 62.4 MB/s 
     |████████████████████████████████| 769 kB 81.0 MB/s 
     |████████████████████████████████| 365 kB 68.8 MB/s 
     |████████████████████████████████| 5.5 MB 75.4 MB/s 
     |████████████████████████████████| 401 kB 62.3 MB/s 
     |████████████████████████████████| 41.4 MB 183 kB/s 
     |████████████████████████████████| 3.6 MB 51.1 MB/s 
     |████████████████████████████████| 60 kB 8.3 MB/s 
     |████████████████████████████████| 212 kB 88.2 MB/s 
     |████████████████████████████████| 132 kB 93.1 MB/s 
     |████████████████████████████████| 182 kB 85.4 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 

In [2]:
import textattack
import pandas as pd
import json

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from datasets import load_dataset

In [6]:
root = '/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project'


In [7]:
from datasets import load_from_disk

In [8]:
dataset = load_from_disk(f"/content/drive/Shareddrives/BipinA/EE8227_secure_ml/Project/data/imdb_test_6k.hf")

In [9]:
test_dataset = HuggingFaceDataset(dataset)

In [10]:
len(test_dataset)

6000

In [11]:
checkpoint_name = "textattack/distilbert-base-uncased-imdb"

model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)


Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [12]:
import torch

# Attack 1: BAEGarg2019

In [13]:
torch.cuda.empty_cache()

In [14]:
from textattack.attack_recipes import BAEGarg2019

In [15]:
attack = BAEGarg2019.build(model_wrapper)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [16]:
import numpy as np

In [17]:
import torch

In [18]:
import os 

In [19]:
torch.cuda.device_count()

1

In [20]:
attack_args = AttackArgs(num_examples=1000,
                        parallel = bool(torch.cuda.device_count()),
                        log_to_csv=f"{root}/data/log_BAEGarg2019.csv",
                        checkpoint_interval=100, 
                        checkpoint_dir=f"{root}/checkpoints_log_BAEGarg2019",
                        csv_coloring_style = 'plain',
                        disable_stdout=True,
                        shuffle=True,
                        enable_advance_metrics=True,
                        silent=True
                        )

attacker = Attacker(attack, test_dataset, attack_args)

attack_results = attacker.attack_dataset()

[Succeeded / Failed / Skipped / Total] 74 / 17 / 9 / 100:  10%|█         | 100/1000 [26:28<3:58:20, 15.89s/it]

[Succeeded / Failed / Skipped / Total] 148 / 39 / 13 / 200:  20%|██        | 200/1000 [43:46<2:55:06, 13.13s/it]

[Succeeded / Failed / Skipped / Total] 210 / 70 / 20 / 300:  30%|███       | 300/1000 [1:03:49<2:28:54, 12.76s/it]

[Succeeded / Failed / Skipped / Total] 283 / 91 / 26 / 400:  40%|████      | 400/1000 [1:24:57<2:07:26, 12.74s/it]

[Succeeded / Failed / Skipped / Total] 352 / 115 / 33 / 500:  50%|█████     | 500/1000 [1:48:32<1:48:32, 13.03s/it]

...

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (228388 > 1024). Running this sequence through the model will result in indexing errors


18 Hours

In [ ]:
%%shell
rm -r checkpoints

In [ ]:
df = pd.read_csv("log.csv")

In [ ]:
df["original_text"].iloc[2]

'I\'m thinking of some things for this movie: First, really is a very bad movie. This is really "Superbad". The film looked very promising in the trailers but fell flat... Maybe the original idea was good, but between a bad script and bad acting the movie became boring and empty. My advise is don\'t waste 2 hours of your precious time. You have been warned. This is the first movie I rated 1 star at IMDb.com... Second, none of the characters are likable. You really don\'t care what happened to them... Third, the villain is very easy to identify. The grandson kills his father, sodomizes the friend\'s son, get the maid pregnant, smothers his grandfather... Like JT says, "If you like evil with no retribution, this is your movie". Nothing more to say...'

In [ ]:
df["original_text"] == df["perturbed_text"]

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [ ]:
df['perturbed_text_clean'] = df.perturbed_text.apply(strip_html)

In [ ]:
t = df[['perturbed_text_clean','ground_truth_output']]

In [ ]:
l = list(t.itertuples(index=False,name=None))

In [ ]:
import json 

In [ ]:
data_set_bipin = textattack.datasets.Dataset(l)

# Evaluate on Perturbed inputs

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length",
                     truncation=True)

In [ ]:
tokenized_datasets_all = data_set_bipin.map(tokenize_function,
                                            batched=True)

AttributeError: ignored

In [ ]:
eval_dataloader = DataLoader(data_set_bipin,
                             batch_size=8)

NameError: ignored

In [ ]:
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    clf_metrics.add_batch(predictions=predictions, references=batch["labels"])


NameError: ignored

In [ ]:
train_preds = []
train_targets = []
for batch in train_dataloader:
   loss, preds, targets = training_step(model, tokenizer, batch)
   train_preds.append(preds)
   train_targets.append(targets)

   # clear gradients
   optimizer.zero_grad()

   # backward
   loss.backward()

   # update parameters
   optimizer.step()
   if scheduler:
      scheduler.step()

# Calculate training accuracy using `train_preds` and `train_targets`

eval_preds = []
eval_targets = []
for batch in eval_dataloader:
   loss, preds, targets = training_step(model, tokenizer, batch)
   eval_preds.append(preds)
   eval_targets.append(targets)



# Calculate eval accuracy using `eval_preds` and `eval_targets`